In [ ]:
import os
import json
import music21 as m21
import keras
import numpy as np

In [ ]:
KERN_DATASET_PATH = "/content/drive/MyDrive/deutschl/essen/europa/deutschl/erk"
DURATION = [
    0.25, 0.5, 0.75, 1.0, 1.5, 2., 3, 4
]
SAVE_DIR = "/content/drive/MyDrive/deutschl/essen/europa/dataset"
SINGLE_FILE_DATASET = "/content/drive/MyDrive/deutschl/essen/europa/deutschl/single_file.txt"
SEQUENCE_LENGTH = 64
MAPPING_PATH = "/content/drive/MyDrive/deutschl/essen/europa/deutschl/mapping.json"

In [ ]:
def load_songs_in_kern(dataset_path):
    songs = []
    for path, subdirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(".krn"):
                song_path = os.path.join(path, file)
                # print(f"Loading song: {song_path}")
                song = m21.converter.parse(song_path)
                songs.append(song)
    return songs

def acceptable_duration(song, duration):
    for note in song.flat.notesAndRests:
        if note.duration.quarterLength not in duration:
            return False

    return True

#didnt understand much, vv technical music wise
def transpose(song):
    parts = song.getElementsByClass(m21.stream.Part)
    measure_part0 = parts[0].getElementsByClass(m21.stream.Measure)
    key = measure_part0[0][4]


    if not isinstance(key, m21.key.Key):
        key = song.analyze("key")
    print(key)
    if key.mode == "major":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C"))
    elif key.mode == "minor":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A"))

    return song.transpose(interval)


def encode_song(song, time_step = 0.25):
    #converting all notes to their midi notation and rests to "r"
    encoded_song = []
    for event in song.flat.notesAndRests:
        if isinstance(event, m21.note.Note):
            symbol = event.pitch.midi
        elif isinstance(event, m21.note.Rest):
            symbol = "r"

        #converting the note/rest to into time/series notation
        steps = int(event.duration.quarterLength / time_step)
        for step in range(steps):
            if step == 0:
                encoded_song.append(symbol)
            else:
                encoded_song.append("_")

    encoded_song = " ".join(map(str, encoded_song))
    return encoded_song

def preprocess(dataset_path):
    #load the songs
    print("Loading....")
    songs = load_songs_in_kern(dataset_path)
    print(f"Loaded {len(songs)} songs")
    #filter out songs with non acceptable durations
    for i, song in enumerate(songs):
        if not acceptable_duration(song, DURATION):
            continue
    #transpose songs to c major or a minor
        song = transpose(song)
    #encode songs with music time series representation
        encoded_song = encode_song(song)
    #save songs to a text file
        save_path = os.path.join(SAVE_DIR, str(i))
        with open(save_path, "w") as fp:
            fp.write(encoded_song)


def load(file_path):
    with open(file_path, "r") as fp:
        song = fp.read()
    return song

def create_single_file(dataset_path, file_dataset_path, sequence_length):
    new_song_delimiter = "/ " * sequence_length
    songs = ""

    for path,_, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(path, file)
            song = load(file_path)
            songs+= song+ " " + new_song_delimiter

    songs = songs[:-1]

    with open(file_dataset_path, "w") as fp:
        fp.write(songs)

    return songs


def create_mapping(songs, mapping_path):
    mappings = {}

    songs = songs.split()
    vocabulary = list(set(songs))

    for i, symbol in enumerate(vocabulary):
        mappings[symbol] = i

    with open(mapping_path, "w") as fp:
        json.dump(mappings, fp, indent=4)

def convert_songs_to_int(songs):
    int_songs = []

    with open(MAPPING_PATH, "r") as fp:
        mappings = json.load(fp)

    songs = songs.split()

    for symbol in songs:
        int_songs.append(mappings[symbol])

    return int_songs

def training_sequences(sequence_length):

    songs = load(SINGLE_FILE_DATASET)
    int_songs = convert_songs_to_int(songs)


    inputs = []
    targets = []
    num_sequences = len(int_songs) - sequence_length
    for i in range(num_sequences):
        inputs.append(int_songs[i:i+sequence_length])
        targets.append(int_songs[i+sequence_length])

    vocabulary_size = len(set(int_songs))
    inputs = keras.utils.to_categorical(inputs, num_classes=vocabulary_size)
    targets = np.array(targets)

    return inputs, targets


def main():
    preprocess(KERN_DATASET_PATH)
    songs = create_single_file(SAVE_DIR, SINGLE_FILE_DATASET, SEQUENCE_LENGTH)
    create_mapping(songs, MAPPING_PATH)
    # inputs, targets = training_sequences(SEQUENCE_LENGTH)

In [ ]:
if __name__ == "__main__":
    main()

Loading....
Loaded 1700 songs
G major
G major
B- major
G major
G major
A major
G major
F major
B- major
G major
g minor
F major
C major
F major
F major
E- major
F major
F major
G major
C major
G major
D major
F major
C major
F major
a minor
G major
G major
G major
G major
F major
D major
G major
G major
G major
A major
d minor
C major
G major
G major
G major
F major
G major
G major
F major
G major
G major
F major
G major
F major
a minor
B- major
D major
D major
C major
D major
C major
B- major
G major
g minor
G major
G major
g minor
C major
F major
G major
a minor
G major
G major
G major
F major
G major
F major
G major
G major
G major
G major
G major
g minor
G major
G major
G major
B- major
G major
F major
C major
D major
G major
G major
G major
F major
G major
F major
F major
F major
B- major
G major
F major
F major
F major
F major
B- major
G major
a minor
F major
D major
G major
a minor
G major
G major
G major
G major
F major
G major
A major
F major
G major
F major
G major
F major
F 

In [ ]:
OUTPUT_UNITS = 38
LOSS = "sparse_categorical_crossentropy"
LEARNING_RATE = 0.01
NUM_UNITS = [256]
EPOCHS = 50
BATCH_SIZE = 64
SAVE_MODEL_PATH = "/content/drive/MyDrive/deutschl/essen/europa/deutschl/model.h5"

In [ ]:
def build_model(output_units, num_units, loss, learning_rate):
    input = keras.layers.Input(shape = (None, output_units))
    x = keras.layers.LSTM(num_units[0])(input)
    x = keras.layers.Dropout(0.2)(x)


    output = keras.layers.Dense(output_units, activation="softmax")(x)
    model = keras.Model(input, output)

    model.compile(loss = loss, optimizer = keras.optimizers.Adam(lr = learning_rate), metrics = ["accuracy"])
    model.summary()
    return model


def train(output_units = OUTPUT_UNITS, num_units = NUM_UNITS, loss = LOSS, learning_rate = LEARNING_RATE):
    inputs, targets = training_sequences(SEQUENCE_LENGTH)
    model = build_model(output_units, num_units, loss, learning_rate)
    model.fit(inputs, targets, epochs = EPOCHS, batch_size= BATCH_SIZE)
    model.save(SAVE_MODEL_PATH)


train()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 38)]        0         
                                                                 
 lstm (LSTM)                 (None, 256)               302080    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 38)                9766      
                                                                 
Total params: 311,846
Trainable params: 311,846
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50
5660/5660 [==============================] - 46s 7ms/step - loss: 0.7144 - accuracy: 0.7811
Epoch 2/50
5660/5660 [==============================] - 39s 7ms/step - loss: 0.7809 - accuracy: 0.7821
Epoch 3/50
5660/5660 [==============================] - 40s 7ms/step - loss: 0.6834 - accuracy: 0.7923
Epoch 4/50
5660/5660 [==============================] - 40s 7ms/step - loss: 0.5886 - accuracy: 0.8095
Epoch 5/50
5660/5660 [==============================] - 40s 7ms/step - loss: 0.5694 - accuracy: 0.8143
Epoch 6/50
5660/5660 [==============================] - 40s 7ms/step - loss: 0.5560 - accuracy: 0.8194
Epoch 7/50
5660/5660 [==============================] - 40s 7ms/step - loss: 0.5479 - accuracy: 0.8217
Epoch 8/50
5660/5660 [==============================] - 40s 7ms/step - loss: 0.5393 - accuracy: 0.8240
Epoch 9/50
5660/5660 [==============================] - 41s 7ms/step - loss: 0.5353 - accuracy: 0.8256
Epoch 10/50
5660/5660 [==============================] - 40s 7ms/step - l

In [ ]:
class MelodyGenerator:
    def __init__(self, model_path="/content/drive/MyDrive/deutschl/essen/europa/deutschl/model.h5"):

        self.model_path = model_path
        self.model = keras.models.load_model(model_path)

        with open(MAPPING_PATH, "r") as fp:
            self._mappings = json.load(fp)

        self._start_symbols = ["/"] * SEQUENCE_LENGTH


    def generate_melody(self, seed, num_steps, max_sequence_length, temperature):
        seed = seed.split()
        melody = seed
        seed = self._start_symbols + seed
        seed = [self._mappings[symbol] for symbol in seed]

        for _ in range(num_steps):
            seed = seed[-max_sequence_length:]
            onehot_seed = keras.utils.to_categorical(seed, num_classes=len(self._mappings))
            onehot_seed = onehot_seed[np.newaxis, ...]
            probabilities = self.model.predict(onehot_seed)[0]
            output_int = self._sample_with_temperature(probabilities, temperature)
            seed.append(output_int)
            output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]

            if output_symbol == "/":
                break

            melody.append(output_symbol)
        return melody

    def _sample_with_temperature(self, probabilites, temperature):
        predictions = np.log(probabilites) / temperature
        probabilites = np.exp(predictions) / np.sum(np.exp(predictions))
        choices = range(len(probabilites)) # [0, 1, 2, 3]
        index = np.random.choice(choices, p=probabilites)
        return index

    def save_melody(self, melody, step_duration=0.25, format="midi", file_name="/content/drive/MyDrive/deutschl/essen/europa/deutschl/mel.mid"):
        stream = m21.stream.Stream()
        start_symbol = None
        step_counter = 1
        for i, symbol in enumerate(melody):
            if symbol != "_" or i + 1 == len(melody):
                if start_symbol is not None:
                    quarter_length_duration = step_duration * step_counter
                    if start_symbol == "r":
                        m21_event = m21.note.Rest(quarterLength=quarter_length_duration)
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)
                    stream.append(m21_event)
                    step_counter = 1
                start_symbol = symbol
            else:
                step_counter += 1
        stream.write(format, file_name)


if __name__ == "__main__":
    mg = MelodyGenerator()
    seed = "67 _ 67 _ 67 _ _ 65 64 _ 64 _ 64 _ _"
    seed2 = "67 _ _ _ _ _ 65 _ 64 _ 62 _ 60 _ _ _"
    melody = mg.generate_melody(seed2, 500, SEQUENCE_LENGTH, 0.3)
    mg.save_melody(melody)

1/1 [==============================] - 0s 19ms/step


<ipython-input-23-65496deb2d20>:35: RuntimeWarning: divide by zero encountered in log
  predictions = np.log(probabilites) / temperature


1/1 [==============================] - 0s 27ms/step
